In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from data import create_image

In [2]:
def create_dataset_structure(output_path):
    """
    Sets up the directory structure for train, test, and optionally val folders.
    Now uses binary classification: no_change (0) and reversal (1)
    """
    for split in ['train', 'test', 'val']:
        for label in ['no_change', 'reversal']:
            (Path(output_path) / split / label).mkdir(parents=True, exist_ok=True)

def load_data(csv_path):
    """
    Reads a CSV file containing OHLC data.
    Assumes columns: Date, Open, High, Low, Close
    """
    return pd.read_csv(csv_path, parse_dates=['Date'])

def classify_outcome(df, lookback=5, min_trend_strength=0.6):
    """
    Classifies each row based on trend continuity and reversal points.
    Labels: 0 for no significant trend/change, 1 for trend reversal point

    Parameters:
    - df: DataFrame with OHLC data
    - lookback: Number of periods to look back for trend identification
    - min_trend_strength: Minimum percentage of consistent price movements to confirm trend

    Returns:
    DataFrame with added classification labels
    """
    # Calculate price changes
    df['price_change'] = df['Close'].diff()
    df['pct_change'] = df['Close'].pct_change()

    # Initialize label column
    df['label'] = 0  # Default to no_change

    # Function to detect trend direction
    def get_trend_direction(window, min_trend_strength=0.001):
        if len(window) < 2:
            return 0

        # Get last two points' direction (primary signal)
        last_move = window.iloc[-1] - window.iloc[-2]

        # Get immediate strength of move
        last_move_strength = abs(last_move) / window.iloc[-2]

        # If last move is too small, consider it noise
        if last_move_strength < min_trend_strength:
            return 0

        # Calculate context from previous points if available
        if len(window) > 2:
            # Calculate recent high and low
            recent_high = max(window.iloc[-3:])
            recent_low = min(window.iloc[-3:])
            price_range = recent_high - recent_low

            # Determine position within recent range
            current_price = window.iloc[-1]
            range_position = (current_price - recent_low) / (price_range if price_range > 0 else 1)

            # Strong trend if price is near the extremes of recent range
            if last_move > 0:  # Potential uptrend
                trend_strength = range_position  # Higher score if near recent high
            else:  # Potential downtrend
                trend_strength = 1 - range_position  # Higher score if near recent low

            # Combine last move with context
            if abs(last_move) > 0:
                # 80% weight to last move, 20% to context
                final_direction = 0.8 * np.sign(last_move) + 0.2 * np.sign(trend_strength - 0.5)

                if abs(final_direction) > 0.5:  # Threshold for trend confirmation
                    return np.sign(final_direction)
                return 0

        # If only 2 points or couldn't confirm with context, just use last move
        return np.sign(last_move) if abs(last_move_strength) > min_trend_strength else 0

    # Function to detect reversal points
    def is_reversal_point(row_idx):
        if row_idx < lookback or row_idx >= len(df) - 1:
            return False

        # Get previous trend
        prev_window = df['price_change'].iloc[row_idx-lookback:row_idx]
        prev_trend = get_trend_direction(prev_window)

        # Get next day's movement
        next_change = df['price_change'].iloc[row_idx+1]

        # If we had a clear trend and the next move is opposite
        if prev_trend != 0:
            return (prev_trend > 0 and next_change < 0) or (prev_trend < 0 and next_change > 0)

        return False

    # Identify reversal points
    for i in range(lookback, len(df) - 1):
        if is_reversal_point(i):
            df.loc[df.index[i], 'label'] = 1  # Mark as reversal point

    # Add additional features that might be useful for visualization
    df['trend_direction'] = df['price_change'].rolling(lookback).apply(get_trend_direction)
    df['trend_strength'] = df['price_change'].rolling(lookback).apply(
        lambda x: max((x > 0).sum(), (x < 0).sum()) / len(x)
    )

    return df.dropna(subset=['label'])

def generate_images(df, output_path, split='train', lookback=4, image_size=64, channels=3):
    """
    Generates images for each lookback window and saves them into structured folders.
    Uses binary classification: no_change (0) and reversal (1)
    """
    for i in tqdm(range(lookback, len(df)), desc=f"Generating {split} images"):
        window_df = df.iloc[i-lookback:i]
        label = int(df['label'].iloc[i])  # Get label for current row
        image_array = create_image([window_df], width=image_size, height=image_size, lookback=lookback, rgb_channels=channels)
        img = Image.fromarray(image_array)
        
        # Map labels to folder names
        label_folder = {0: "no_change", 1: "reversal"}[label]
        img_save_path = Path(output_path) / split / label_folder / f"{i}_{label_folder}.png"
        img.save(img_save_path)

def make_dataset(csv_path, output_path):
    """
    Main function to create dataset from a CSV file and save images to output_path.
    """
    create_dataset_structure(output_path)  # Initialize dataset folders
    df = load_data(csv_path)  # Load OHLC data
    df = classify_outcome(df)  # Classify data based on trend reversals
    
    # Split data (e.g., 80% train, 10% test, 10% val)
    train_size = int(0.7 * len(df))
    test_size = int(0.2 * len(df))
    train_df = df.iloc[:train_size]
    test_df = df.iloc[train_size:train_size + test_size]
    val_df = df.iloc[train_size + test_size:]
    
    # Generate and save images for each dataset split
    generate_images(train_df, output_path, split='train')
    generate_images(test_df, output_path, split='test')
    if not val_df.empty:
        generate_images(val_df, output_path, split='val')

In [3]:
import yaml

def create_yolo_yaml(output_path, yaml_filename="data.yaml"):
    """
    Creates a YOLO-compatible YAML file that specifies the dataset structure.
    """
    yaml_data = {
        'train': str(Path(output_path) / 'train'),  # Path to training data
        'val': str(Path(output_path) / 'val'),      # Path to validation data
        'test': str(Path(output_path) / 'test'),    # Path to test data
        'nc': 2,                                    # Number of classes
        'names': ['no_change', 'reversal']       # Class names
    }

    yaml_path = Path(output_path) / yaml_filename
    with open(yaml_path, 'w') as yaml_file:
        yaml.dump(yaml_data, yaml_file, default_flow_style=False)
    print(f"YOLO YAML file created at: {yaml_path}")

In [4]:
make_dataset("./data/unprocessed/SPY.csv", "./data/processed/SPY_dataset")

Generating val images: 100%|██████████| 791/791 [00:29<00:00, 26.61it/s]


In [5]:
create_yolo_yaml("./data/processed/SPY_dataset")

YOLO YAML file created at: data\processed\SPY_dataset\data.yaml


In [6]:
# %load_ext tensorboard
# %tensorboard --logdir runs/SPY

Reusing TensorBoard on port 6006 (pid 64112), started 2 days, 16:03:39 ago. (Use '!kill 64112' to kill it.)

In [14]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11l-cls.pt", task="classify", verbose=True)  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="./data/processed/SPY_dataset",
                      epochs=100, imgsz=64, device="0",
                      batch=64, workers=6,
                      dynamic=False, pretrained=True, name="SPY",
                      project="./runs", multi_scale=False,
                      save_dir="./runs", auto_augment=None)

100%|██████████| 27.2M/27.2M [00:02<00:00, 11.2MB/s]


New https://pypi.org/project/ultralytics/8.3.27 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.24  Python-3.11.1 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
engine\trainer: task=classify, mode=train, model=yolo11l-cls.pt, data=./data/processed/SPY_dataset, epochs=100, time=None, patience=100, batch=128, imgsz=64, save=True, save_period=-1, cache=False, device=0, workers=6, project=./runs, name=SPY10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=N

train: Scanning E:\Desktop\Coding\Imperial\Assignments\Innovative-Approaches-to-Asset-Prediction\data\processed\SPY_dataset\train... 5553 images, 0 corrupt: 100%|██████████| 5553/5553 [00:00<?, ?it/s]
val: Scanning E:\Desktop\Coding\Imperial\Assignments\Innovative-Approaches-to-Asset-Prediction\data\processed\SPY_dataset\val... 791 images, 0 corrupt: 100%|██████████| 791/791 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 82 weight(decay=0.0), 83 weight(decay=0.001), 83 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 64 train, 64 val
Using 6 dataloader workers
Logging results to runs\SPY10
Starting training for 100 epochs...

      Epoch    GPU_mem       loss  Instances       Size


      1/100      1.25G      0.662         49         64: 100%|██████████| 44/44 [00:05<00:00,  8.40it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 4/4 [00:00<00:00, 24.91it/s]

...

    100/100      1.31G      0.257         49         64: 100%|██████████| 44/44 [00:05<00:00,  8.16it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 4/4 [00:00<00:00, 44.19it/s]

                   all      0.714          1



100 epochs completed in 0.194 hours.
Optimizer stripped from runs\SPY10\weights\last.pt, 25.9MB
Optimizer stripped from runs\SPY10\weights\best.pt, 25.9MB

Validating runs\SPY10\weights\best.pt...
Ultralytics 8.3.24  Python-3.11.1 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
YOLO11l-cls summary (fused): 227 layers, 12,820,994 parameters, 0 gradients, 49.3 GFLOPs
train: E:\Desktop\Coding\Imperial\Assignments\Innovative-Approaches-to-Asset-Prediction\data\processed\SPY_dataset\train... found 5553 images in 2 classes  
val: E:\Desktop\Coding\Imperial\Assignments\Innovative-Approaches-to-Asset-Prediction\data\processed\SPY_dataset\val... found 791 images in 2 classes  
test: E:\Desktop\Coding\Imperial\Assignments\Innovative-Approaches-to-Asset-Prediction\data\processed\SPY_dataset\test... found 1583 images in 2 classes  


               classes   top1_acc   top5_acc: 100%|██████████| 4/4 [00:00<00:00, 25.63it/s]


                   all      0.742          1
Speed: 0.0ms preprocess, 0.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\SPY10


In [15]:
metrics = model.val()
print(metrics)

Ultralytics 8.3.24  Python-3.11.1 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
YOLO11l-cls summary (fused): 227 layers, 12,820,994 parameters, 0 gradients, 49.3 GFLOPs
train: E:\Desktop\Coding\Imperial\Assignments\Innovative-Approaches-to-Asset-Prediction\data\processed\SPY_dataset\train... found 5553 images in 2 classes  
val: E:\Desktop\Coding\Imperial\Assignments\Innovative-Approaches-to-Asset-Prediction\data\processed\SPY_dataset\val... found 791 images in 2 classes  
test: E:\Desktop\Coding\Imperial\Assignments\Innovative-Approaches-to-Asset-Prediction\data\processed\SPY_dataset\test... found 1583 images in 2 classes  


val: Scanning E:\Desktop\Coding\Imperial\Assignments\Innovative-Approaches-to-Asset-Prediction\data\processed\SPY_dataset\val... 791 images, 0 corrupt: 100%|██████████| 791/791 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 7/7 [00:00<00:00, 16.92it/s]


                   all      0.742          1
Speed: 0.0ms preprocess, 0.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\SPY102
ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000245E0470DD0>
curves: []
curves_results: []
fitness: 0.87104931473732
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.7420986294746399, 'metrics/accuracy_top5': 1.0, 'fitness': 0.87104931473732}
save_dir: WindowsPath('runs/SPY102')
speed: {'preprocess': 0.03358748106829888, 'inference': 0.45782395470157433, 'loss': 0.0012520743079492927, 'postprocess': 0.0}
task: 'classify'
top1: 0.7420986294746399
top5: 1.0


In [16]:
!taskkill /IM "tensorboard.exe" /F

ERROR: The process "tensorboard.exe" not found.
